In [1]:
def ie(A,r):
    k=[]
    n=A.nrows()
    for i in range(0,n):
        l= [0]*(n+1)
        l[0]=r[i]
        l[1:]=A[i]
        k.append(l)
    return k


ie(Matrix([[1,0],[0,1],[-1,0],[0,-1],[1,1]]),[-3,-3,-3,-3,-3])

[[-3, 1, 0], [-3, 0, 1], [-3, -1, 0], [-3, 0, -1], [-3, 1, 1]]

In [2]:
def ie2(A,r):
    k=[]
    n=A.nrows()
    for i in range(0,n):
        l= [0]*(n+1)
        l[0]=r[i]
        l[1:]=-A[i]
        k.append(l)
    return k



In [3]:
%%time
ie2(Matrix([[1,0],[0,1],[-1,0],[0,-1],[1,1]]),[-3,-3,-3,-3,-3])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 470 µs


[[-3, -1, 0], [-3, 0, -1], [-3, 1, 0], [-3, 0, 1], [-3, -1, -1]]

In [4]:
def CheckR(A,R):
    for r in R:
        if Polyhedron(ieqs=ie2(A,r)).dim()<=0:
            return False
    return True

In [5]:
def find_intersection(S):
    if len(S) == 1:
        return S[0]
    K=S[0]
    for i in range(1,len(S)):
        K=K.intersection(S[i])
    return K

In [6]:
list(range(1,2))

[1]

In [7]:
def find_Is(subset,m):
    Im = Matrix.identity(m)
    for s in subset:
        Im[s-1,s-1]= 0
    return [x[:] for x in Im if x != 0]

find_Is([1],5)

[(0, 1, 0, 0, 0), (0, 0, 1, 0, 0), (0, 0, 0, 1, 0), (0, 0, 0, 0, 1)]

In [8]:
def foo(list,i):
    print(list[:i]+list[i+1:])
    
foo([1,0,0],0)

[0, 0]


In [9]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [10]:
def ChamberComplex(A):
    m=A.nrows()
    d=A.ncols()
    if A.rank() == d:
        S = Subsets([x for x in range(1,m+1)],d,submultiset=True)
        S = [sub for sub in S]
       
        Cones=[]
        for s in S:
            Is=find_Is(s,m)
            CAs=Polyhedron(rays=Is,lines=A.columns())
            Cones.append(CAs)
        
        ChamberComplex=[]
        RightKernel = Polyhedron(lines= A.transpose().right_kernel().gens())# [[1,0,1,0,0],[0,1,0,1,0],[-1,-1,0,0,1]]
        
        Pcones = list(powerset(Cones))
       
        
        for d in Pcones:
            if len(d)!=0:
                if len(d)!=len(Cones):
                    K = find_intersection(d)
                    
                    KK= K.intersection(RightKernel)
                    #print(KK.rays())
                    R = list(K.rays())
                    
                    if CheckR(A,R):
                        ChamberComplex.append(KK)
    
        ll=len(ChamberComplex)
        for i in range(0,ll):
            if ChamberComplex[i] == None:
                continue
            for j in range(0,ll):
                if i==j or ChamberComplex[j] == None:
                    continue
                if dim(ChamberComplex[i])==dim(ChamberComplex[j]) and ChamberComplex[i].intersection(ChamberComplex[j]) == ChamberComplex[i]:
                    ChamberComplex[j]=None
             
        return [c for c in ChamberComplex if c != None]
        

In [11]:
def last(ChamberComplex):
    ll=len(ChamberComplex)
    for i in range(0,ll):
        if ChamberComplex[i] == None:
            continue
        for j in range(0,ll):
            if i==j or ChamberComplex[j] == None:
                continue
            if ChamberComplex[i].intersection(ChamberComplex[j]) == ChamberComplex[i]:
                ChamberComplex[j]=None
                    
    return [c for c in ChamberComplex if c != None]

In [12]:
def isZero(vertex):
    for i in vertex[0]:
        if i!=0:
            return False
    return True

In [13]:
def helper(L,first,last,Output,inter,Rk,A,ChamberComplex):
    
    if first==last:
        return

    if len(Output)+1 == last:
        return
    
    for i in range(first ,last):
        if inter !=0 :
            
            K = inter.intersection(L[i])
            KK= K.intersection(Rk)
            if dim(KK)==0:
                continue
            if CheckR(A,list(K.rays())):
                try:
                    ChamberComplex[dim(KK)].append(KK)
                except:
                    ChamberComplex[dim(KK)]=[KK]
            
        else:
            K=L[i]
            KK = K.intersection(Rk)
            
            if dim(KK)==0:
                continue
            if CheckR(A,list(K.rays())):
                try:
                    ChamberComplex[dim(KK)].append(KK)
                except:
                    ChamberComplex[dim(KK)]=[KK]
        
            

        helper(L,i+1,last,Output+[L[i]],K,Rk,A,ChamberComplex)

In [14]:
def powerset_f(L,Rk,A,ChamberComplex):
    helper(L,0,len(L),[],0,Rk,A,ChamberComplex)

In [15]:
def ChamberComplex_f(A):
    m=A.nrows()
    d=A.ncols()
    if A.rank() == d:
        S = Subsets([x for x in range(1,m+1)],d,submultiset=True)
        S = [sub for sub in S]
        Cones=[]
        for s in S:
            Is=find_Is(s,m)
            CAs=Polyhedron(rays=Is,lines=A.columns())
            Cones.append(CAs)
        
        #Cones=cc;
        ChamberComplex={}
        RightKernel = Polyhedron(lines=A.transpose().right_kernel().gens())
        
       
        powerset_f(Cones,RightKernel,A,ChamberComplex)
        
        ret = []
        for key in ChamberComplex:
            ret+=last(ChamberComplex[key])
            
        return ret

In [16]:
%%time


l=ChamberComplex_f(Matrix([[93,-78,-64,-34],[ 41, -12, -15, -15],[98 ,38, -97, 46],[-58, 75, 18, 54],[-66,70,37,-59],[4,61,58,-28]]))

CPU times: user 7.61 s, sys: 47 ms, total: 7.66 s
Wall time: 7.9 s


In [17]:
%%time
k=ChamberComplex(Matrix([[93,-78,-64,-34],[ 41, -12, -15, -15],[98 ,38, -97, 46],[-58, 75, 18, 54],[-66,70,37,-59],[4,61,58,-28]]))

CPU times: user 2min 37s, sys: 234 ms, total: 2min 37s
Wall time: 2min 37s
